In [2]:
from sage.all import * 
import struct

DR = RealField(53)

DD = RealField(190)

def double_to_hex(f):
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x = RealField(190)(x).exact_rational()
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def split_triple_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_mid = DR(x - DD(x_hi))
    x_lo = x - DD(x_hi) - DD(x_mid)
    return (x_lo, x_mid, x_hi)

def print_double_double(mark, x):
    splat = split_double_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}),")

def print_triple_double(mark, x):
    splat = split_triple_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}, {double_to_hex(splat[2])}),")

In [4]:
from sage.all import *
from mpmath import mp, besseli, taylor

# High precision
mp.prec = 450

# Step 1: Define I0(x) - 1
def shifted_i0(x):
    return besseli(0, x) - mp.mpf(1)

# Step 2: Series of I0(x) - 1 in terms of (x/2)^2
terms = 100
from mpmath import taylor
coeffs = taylor(shifted_i0, 0, terms)

# print(coeffs)

# Step 3: Build series in terms of y = (x/2)^2
R = PolynomialRing(RealField(450), 'y')
y = R.gen()
f = R(0)

for n in range(2, terms, 2):
    k = n // 2
    c = RealField(450)(coeffs[n])
    if n >= 1:
        f += R(c) * y**(k-1) * (4**k)
    else:
        f += R(c) * y**(k-1) * (4**k)

print(f)

rat_approx = SR(f).power_series(RealField(450)).pade(16, 0)
numer = rat_approx.numerator()
denom = rat_approx.denominator()

print(numer)
print(denom)

def I0_approx(y):
    z = (y/2)**2
    return 1 + z * numer(z) / denom(z)

# Print result
print("Rational approximation of I0(x):")
pretty_print(I0_approx)

# Evaluate at some point
print("I0_approx(1) =", I0_approx(-8.5))
print("I0_real(1) =", besseli(0, -8.5))

print("Numerator polynomial coefficients:")

coeffs = numer.coefficients(sparse=False) 

for i, c in enumerate(coeffs):
    print("f64::from_bits(" + double_to_hex(c) + "),")


2.70265284158458111246639178380860921965281806355772639932174388328913237632303651273240661304359005444739555445283608842518250882276959e-126*y^48 + 6.48906947264457925103180667292447073638641617060210108477150706377720683555161066707050827791765972072819672624125944830886320368346979e-123*y^47 + 1.49508160649731105943772825744179805766343028570672408993135522749426845491109109769304510723222879965577652572598617689036208212867144e-119*y^46 + 3.30263526875256013029794172068893190937851750112615351465836369753483901689860023480393664187599341843961034532870346475080983942223518e-116*y^45 + 6.98837622868041723571044468097777992024494303238294083701709758398371935975743809684512993420960207341821549071553653141271362021744968e-113*y^44 + 1.41514618630778449023136504789800043384960096405754551949596226075670317035088121461113881167744441986718863686989614761107450809403358e-109*y^43 + 2.73972301669187077308792273273052883993282746641540812574418293682497733779930603148716473940753239686287

In [200]:
from mpmath import mp, besseli

# Setup precision
mp.dps = 100

# Define the interval for y (example: from 0 to 3)
a = 0.0
b = 7.5

# Number of test points
N = 1000

# Use your RealField 300 for evaluation
RF = RealField(53)

# Prepare to store max relative error
max_rel_error = 0

for i in range(N+1):
    y_val = a + (b - a) * i / N
    y_rf = RF(y_val)
    
    approx_val = I0_approx(y_rf)
    true_val = mp.besseli(0, y_val)
    
    # Compute relative error
    rel_err = abs(approx_val - true_val) / abs(true_val)
    
    if rel_err > max_rel_error:
        max_rel_error = rel_err

print(f"Maximum relative error on [{a}, {b}] is about {-RealField(70)(max_rel_error).log2()}")


Maximum relative error on [0.0, 7.5] is about 44.373857022462585146


In [197]:
from mpmath import mp, besseli, nprint

# High precision
mp.prec = 350

# Define the function to expand
def f(x):
    return mp.sqrt(x) * mp.exp(-x) * besseli(0, x)

# Generate coefficients of series in u = 1/x
terms = 30
coeffs = []

for n in range(terms):
    u = mp.mpf('1e-27')  # Very small u ≈ 1/x
    deriv = mp.diff(lambda u: f(1/u), u, n)
    coeff = deriv / mp.fac(n)
    coeffs.append(coeff)
    print(f"u^{n}: {coeff}")

x = var('x')
u = x
coeffs = [RR(coeffs[k]) for k in range(terms)]

# Construct polynomial approximation in 1/x
# P_ring = PolynomialRing(RR, 'u')
taylor_poly = sum(c * u**k for k, c in enumerate(coeffs))

print(taylor_poly)

pade_approximant = SR(taylor_poly).power_series(RR).pade(25, 0)

expr_in_x = pade_approximant.subs(u = 1/x)

numer_asympt = expr_in_x.numerator()

print(numer_asympt)
print(denom)

print(f(17.1))
print(numer_asympt(1/17.1))
print(taylor_poly(x=1/17.1))
print(expr_in_x(x=1/17.1))

coeffs = numer_asympt.coefficients(sparse=False) 

for i, c in enumerate(coeffs):
    print("f64::from_bits(" + double_to_hex(c) + "),")


u^0: 0.3989422804014326779399460599842496535260377159074279151857741923208659899625711280778486930461872946378
u^1: 0.04986778505017908474249325754789899174093379923092174697417139487176778314389559541069692253826792737632
u^2: 0.028050629090725735167652457426794441035726732402698397570389399641081804023295032420149456191039220173357
u^3: 0.029219405302839307466304643240569091987399935318391427280347329328272064261664577560878230688342532173713
u^4: 0.04474221436997268955777898514109027958559690918740281381390042662429097603654125305500333888070646372967
u^5: 0.090602984099194696354502445363722736656807222877002925476947114448385232039935411995380932943420719345142
u^6: 0.22839502241671996372697491572475453315602132336429437140075380755137032948594264370588900683313591969677
u^7: 0.68926354979331560481890644692276477897048787031387636612939700526515838644736917148315170278374225263234
u^8: 2.42319216724212517319146799684838226400512280460396561822962797948719082995664827632656037188712

In [198]:
from mpmath import mp, besseli

# Setup precision
mp.dps = 100

# Define the interval for y (example: from 0 to 3)
a = 15.0
b = 20.0

# Number of test points
N = 1000

# Use your RealField 300 for evaluation
RF = RealField(53)

# Prepare to store max relative error
max_rel_error = 0

def I0_approx_asympt(y):
    return numer_asympt(x=1/y) * y.exp() / y.sqrt()

for i in range(N+1):
    y_val = a + (b - a) * i / N
    y_rf = RF(y_val)
    
    approx_val = I0_approx_asympt(y_rf)
    true_val = mp.besseli(0, y_val)
    
    # Compute relative error
    rel_err = abs(approx_val - true_val) / abs(true_val)
    
    if rel_err > max_rel_error:
        max_rel_error = rel_err

print(f"Maximum relative error on [{a}, {b}] is about {-RealField(70)(max_rel_error).log2()}")

Maximum relative error on [15.0, 20.0] is about 43.635221852039236487
